# Análisis de Sentimientos

**Maestría en Inteligencia Artificial**

**Asignatura:** Procesamiento del Lenguaje Natural

Actividad grupal desarrollada por Edmilson Prata da Silva, Mariana Carmona Cruz, Gerardo Davila y Pedro Luis Cabrera en 06/01/2026.


## 💡 Introduccíon

Este trabajo fue desarrollado para atender el requisito de la asignatura de Procesamiento del Lenguaje Natural, de la Maestría en Inteligencia Artificial, de Universidad Internacional de La Rioja, en 2026, hecho por el grupo 1030H.

El objetivo de esta actividad es aplicar los conocimientos adquiridos para resolver un problema de análisis de sentimientos, también conocido como minería de opiniones.


## Rubrix vs Argilla

La construcción del Analizador de Sentimientos comenzó siguiendo las instrucciones del tutorial de Rubrix, disponible en [este enlace](https://rubrix.readthedocs.io/en/master/tutorials/01-labeling-finetuning.html), de acuerdo con las instrucciones de la guía proporcionada para este trabajo.

Sin embargo, Rubrix era una herramienta gratuita y de código abierto para explorar, etiquetar y monitorizar datos en proyectos de PLN (procesamiento del lenguaje natural). Rubrix ha sido reemplazada por una herramienta equivalente llamada Argilla, cuyo tutorial equivalente está disponible en [este enlace](https://docs.v1.argilla.io/en/v1.14.0/tutorials/notebooks/training-textclassification-transformers-pretrained.html). Ambos tutoriales están desactualizados y no son compatibles con las versiones actuales de Argilla.

Para llevar a cabo esta tarea, se adaptaron algunos puntos utilizando los recursos de la versión actual.


## 📚 Librarias

Instalaciones y importaciones de librarias necesárias en el notebook.

In [1]:
# Instalación de las bibliotecas necesarias

#!pip install transformers[torch] datasets scikit-learn ipywidgets -qqq
#!pip install accelerate
#!pip install argilla tf_keras
#!pip install evaluate -qqq

In [2]:
# Importaciones de librarias

import argilla as rg
import numpy as np
import pandas as pd

from evaluate import load
from typing import Iterable, Optional
from datasets import Dataset as HFDataset
from datasets import load_dataset, concatenate_datasets
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

## 📦 Carga de Datos (dataset Banking 77)

Carga los datos del dataset Banking 77. Este conjunto de datos contiene consultas de usuarios de banca en línea anotadas con sus intenciones correspondientes. Etiquetaremos el sentimiento de estas consultas. Esto podría ser útil para asistentes digitales y análisis de atención al cliente.

Los datos serán divididos en dos subconjuntos del 50%. Comenzaremos con la división to_label1 para la exploración y anotación de datos, y conservaremos to_label2 para iteraciones posteriores.

In [3]:
# Carga del dataset Banking 77:
banking_ds = load_dataset("banking77")

# División del dataset del 50%:
to_label1, to_label2 = banking_ds['train'].train_test_split(test_size=0.5, seed=42).values()

## 💹 Carga del modelo Sentiment Distilbert fine-tuned on sst-2

El tutorial de Rubrix explica que, a diciembre de 2021, el modelo distilbert-base-uncased-finetuned-sst-2-english se encuentra entre los cinco modelos de clasificación de texto más populares en Hugging Face Hub.

Este modelo es una optimización del modelo Distilbert que utiliza el SST-2 (Stanford Sentiment Treebank). SST-2 es un bachmark de clasificación de sentimientos muy popular.

El artículo de Rubrix explica que se trata de un clasificador de sentimientos de propósito general que requiere un mayor ajuste para casos de uso y estilos de texto específicos.

En el ejemplo, utilizando el dataset Banking 77, explorase su calidad en consultas de usuarios bancarios y crease un conjunto de entrenamiento para adaptarlo al dominio del banco.

In [4]:
# Carga del modelo distilbert-base-uncased-finetuned-sst-2-english:
sentiment_classifier = pipeline(
    model="distilbert-base-uncased-finetuned-sst-2-english",
    task="sentiment-analysis",
    return_all_scores=True,
)

Device set to use mps:0
/Users/edprata/anaconda3/envs/Python-3-13-2/lib/python3.13/site-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [5]:
#  Prueba con un ejemplo del dataset Banking 77:
to_label1[3]['text'], sentiment_classifier(to_label1[3]['text'])

('Hi, Last week I have contacted the seller for a refund as directed by you, but i have not received the money yet. Please look into this issue with seller and help me in getting the refund.',
 [[{'label': 'NEGATIVE', 'score': 0.9934700727462769},
   {'label': 'POSITIVE', 'score': 0.006529913283884525}]])

📌 Los resultados que aparecen (score) son medidas de probabilidad que indican una mayor probabilidad de que el comentario o informe del cliente sea negativo (0.994) que positivo (0.006).

## 🛠️ Ejecución del modelo preentrenado

Abajo la ejecución del modelo sobre el dataset y el registro de las predicciones. Es utilizado el práctico método dataset.map de la biblioteca datasets.

### 🧪 Predicciones en batch

Realiza predicciones por lotes. Esto está limitado a 100 ejemplos para facilitar las pruebas y reducir el tiempo de ejecución.

In [6]:
test_limit = range(100) # len(to_label1)

def predict(examples):
    return {"predictions": sentiment_classifier(examples['text'], truncation=True)}

to_label1 = to_label1.map(predict, batched=True, batch_size=4).select(test_limit)

to_label1

Dataset({
    features: ['text', 'label', 'predictions'],
    num_rows: 100
})

### 📈 Mostrando resultados

Mostrando resultados de predicciones para verificación:

In [7]:
for element in to_label1:
    print(element['text'], element['predictions'])

Should my cash withdrawal still be pending? [{'label': 'NEGATIVE', 'score': 0.9991445541381836}, {'label': 'POSITIVE', 'score': 0.0008554730447940528}]
Where can I find the auto-top option? [{'label': 'NEGATIVE', 'score': 0.9992862343788147}, {'label': 'POSITIVE', 'score': 0.0007138242362998426}]
My top-up was just cancelled. [{'label': 'NEGATIVE', 'score': 0.9995952248573303}, {'label': 'POSITIVE', 'score': 0.00040481286123394966}]
Hi, Last week I have contacted the seller for a refund as directed by you, but i have not received the money yet. Please look into this issue with seller and help me in getting the refund. [{'label': 'NEGATIVE', 'score': 0.9934700727462769}, {'label': 'POSITIVE', 'score': 0.006529913283884525}]
Why couldn't I make a withdrawal from the ATM? [{'label': 'NEGATIVE', 'score': 0.9984630346298218}, {'label': 'POSITIVE', 'score': 0.0015369340544566512}]
Do EU transfers happen quickly? I purchased something a few days ago and the seller hasn't received my money yet

📌 Arriba se encuentran los resultados de la predicción, etiquetados para indicar si el texto es una reseña o informe positivo o negativo.

#### 📝 Lista de resultados

El siguiente código crea una lista de registros de Argilla con las predicciones.

En el tutorial de Argilla, hay código adicional utilizado para registrar información en el servidor de Argilla, pero ha sido eliminado de este trabajo y reemplazado por una alternativa que utiliza datos en memoria, evitando configurar un servidor y haciendo posible realizar el trabajo en un Notebook. El código que registra logs en el servidor Argilla sigue el siguiente ejemplo:

```
client = rg.Argilla(api_url="https://...", api_key="owner.key")
rg.log(name="labeling_with_pretrained", records=records)
```

In [8]:
# Crear registros utilizando la API actual de Argilla (rg.Record):
records = []
for example in to_label1.shuffle():
    record = rg.Record(
        fields={"text": example["text"]},
        metadata={
            "category": example["label"],
            "prediction_agent": "distilbert-base-uncased-finetuned-sst-2-english",
            "predictions": example["predictions"],
        },
    )
    records.append(record)

# Vista previa de los dos primeros registros:
records[:2]

[Record(id=93e0265c-8968-4eba-9267-d391b47f85b1,status=pending,fields={'text': "Why isn't the transfer I made to a friend showing?"},metadata={'category': 66, 'prediction_agent': 'distilbert-base-uncased-finetuned-sst-2-english', 'predictions': [{'label': 'NEGATIVE', 'score': 0.9984500408172607}, {'label': 'POSITIVE', 'score': 0.001549929496832192}]},suggestions={},responses={}),
 Record(id=17668ff9-8afd-4479-9e2c-8be274b09ba2,status=pending,fields={'text': "I think my account has been hacked there are charges on there I don't recognize."},metadata={'category': 22, 'prediction_agent': 'distilbert-base-uncased-finetuned-sst-2-english', 'predictions': [{'label': 'NEGATIVE', 'score': 0.9994450211524963}, {'label': 'POSITIVE', 'score': 0.0005549690104089677}]},suggestions={},responses={})]

## ⚙️ Transformación del Dataset: Argilla x Transformers

Es necesario convertir los datos de un objeto del framework Argilla a un objeto que el framework Hugging Face Transformers pueda usar. Si tuviéramos el servidor Argilla, podríamos ejecutar una consulta. Sin embargo, estamos trabajando en un notebook en Google Colab y esta etapa (el servidor Argilla) no es importante para el objetivo principal de este trabajo. Para ello, se utilizará una conversión manual para obtener un objeto en un formato adecuado.

In [9]:
# Conjunto de datos mínimo local similar a Argilla (sin persistencia del servidor)

class LocalArgillaDataset:
    def __init__(self, name: str, records: Iterable):
        self.name = name
        self._records = [r.to_dict() if hasattr(r, "to_dict") else r for r in records]

    @property
    def records(self):
        return self

    def log(self, records, **kwargs):
        self._records.extend([r.to_dict() if hasattr(r, "to_dict") else r for r in records])
        return self

    def to_pandas(self):
        rows = []
        for r in self._records:
            row = {"id": r.get("id")}
            row.update(r.get("fields", {}))
            row.update({f"metadata.{k}": v for k, v in (r.get("metadata") or {}).items()})
            rows.append(row)
        return pd.DataFrame(rows)

    def prepare_for_training(self, text_field="text", label_field=None):
        df = self.to_pandas()
        if label_field is not None and label_field in df.columns:
            df = df[[text_field, label_field]].dropna()
            # Rename the label column to 'labels' for Hugging Face Trainer compatibility
            df = df.rename(columns={label_field: "labels"})
            # If labels are strings, convert to integer ids
            if df["labels"].dtype == object:
                uniques = sorted(df["labels"].unique())
                mapping = {v: i for i, v in enumerate(uniques)}
                df["labels"] = df["labels"].map(mapping)
            return HFDataset.from_pandas(df.reset_index(drop=True))
        else:
            df = df[[text_field]].dropna()
            return HFDataset.from_pandas(df.reset_index(drop=True))


def make_local_dataset(records, status_filter: Optional[str] = None, name: str = "myDataSet"):
    if status_filter is not None:
        sf = status_filter.lower()
        filtered = [r for r in records if getattr(r, "status", "").lower() == sf]
    else:
        filtered = list(records)
    return LocalArgillaDataset(name=name, records=filtered)

# Crear un conjunto de datos local a partir de `records` (sin persistencia en el servidor Argilla)
rb_dataset = make_local_dataset(records=records)

# Vista previa de las primeras tres filas del conjunto de datos local
rb_dataset.to_pandas().head(3)

,id,text,metadata.category,metadata.prediction_agent,metadata.predictions
0,93e0265c-8968-4eba-9267-d391b47f85b1,Why isn't the transfer I made to a friend show...,66,distilbert-base-uncased-finetuned-sst-2-english,"[{'label': 'NEGATIVE', 'score': 0.998450040817..."
1,17668ff9-8afd-4479-9e2c-8be274b09ba2,I think my account has been hacked there are c...,22,distilbert-base-uncased-finetuned-sst-2-english,"[{'label': 'NEGATIVE', 'score': 0.999445021152..."
2,64dcb3ed-42c2-4f8a-90e2-4af5ad05a864,Let me know when I get charged extra for payme...,15,distilbert-base-uncased-finetuned-sst-2-english,"[{'label': 'NEGATIVE', 'score': 0.997278034687..."


## 🚀 Fine-tune del modelo preentrenado

Ajustaremos nuestro conjunto de entrenamiento de Argilla con la API Trainer de Hugging Face Transformers. Para ello, seguimos de cerca la guía "Ajuste de un modelo preentrenado" de la documentación de Transformers.

### ⚙️ Preparación de Datos

La preparación de los datos inclye la tokenización y la separación de los datos en subconjunto para entrenamiento y evaluacíon.

In [10]:
# Creación del conjunto de datos con etiquetas como IDs numéricos.
# Especificación del campo de etiqueta para asegurar que las etiquetas se incluyan en el conjunto de datos.
train_ds = rb_dataset.prepare_for_training(label_field="metadata.category")

# Tokenización del conjunto de datos
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train_ds = train_ds.map(tokenize_function, batched=True)

# Separación de los datos en un conjunto de entrenamiento y evaluación
train_dataset, eval_dataset = tokenized_train_ds.train_test_split(
    test_size=0.2, seed=42
).values()

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

### ⚙️ Entrenamiento del Clasificador de Sentimientos

Entrenamiento perfeccionamiento (fine-tune) del modelo distilbert-base-uncased-finetuned-sst-2-english.

In [11]:
# Carga del modelo distilbert-base-uncased-finetuned-sst-2-english:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english"
)

# Configuración de los argumentos de entrenamiento:
training_args = TrainingArguments(
    "distilbert-base-uncased-sentiment-banking",
    eval_strategy="epoch",
    logging_steps=30,
)

# Definición de la métrica de precisión:
metric = load("accuracy")

# Función para calcular las métricas durante la evaluación:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Configuración del entrenador:
trainer = Trainer(
    args=training_args,
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

# Verificación de las configuraciones y amostra de los datos tokenizados:
print("Colunas del conjunto de datos:", train_dataset.column_names)
print("Primeros tres ejemplos del conjunto de datos:", train_dataset[:3])

Colunas del conjunto de datos: ['text', 'labels', 'input_ids', 'attention_mask']
Primeros tres ejemplos del conjunto de datos: {'text': ['Can I change my card PIN?', 'Online banking is not showing my cheque or cash deposit so the balance is incorrect.', 'I was under the impression ATM cash withdrawals were free. Why was I suddenly charged for my most recent ATM transaction?'], 'labels': [21, 6, 19], 'input_ids': [[101, 2064, 1045, 2689, 2026, 4003, 9231, 1029, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [12]:
# Entrenamiento del modelo:
trainer.train()
print("Entrenamiento completado.")

/Users/edprata/anaconda3/envs/Python-3-13-2/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.000000,0.000000
2,No log,0.000000,0.000000
3,0.001700,0.000000,0.000000


/Users/edprata/anaconda3/envs/Python-3-13-2/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/edprata/anaconda3/envs/Python-3-13-2/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/edprata/anaconda3/envs/Python-3-13-2/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Entrenamiento completado.


### 🧪 Prueba de modelo personalizado

Probar el modelo personalizado que se entrenó en los pasos anteriores.

In [13]:
# Pipeline para el clasificador de sentimientos ajustado:
finetuned_sentiment_classifier = pipeline(
    model=model.to("cpu"),
    tokenizer=tokenizer,
    task="sentiment-analysis",
    return_all_scores=True,
)

Device set to use mps:0
/Users/edprata/anaconda3/envs/Python-3-13-2/lib/python3.13/site-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [14]:
# Prueba con un ejemplo nuevo:
print('Finetuned', finetuned_sentiment_classifier("I need to deposit my virtual card, how do i do that.")[0])
print('Original', sentiment_classifier("I need to deposit my virtual card, how do i do that.")[0])

Finetuned [{'label': 'NEGATIVE', 'score': 0.9801008701324463}, {'label': 'POSITIVE', 'score': 0.019899118691682816}]
Original [{'label': 'NEGATIVE', 'score': 0.9992493987083435}, {'label': 'POSITIVE', 'score': 0.0007506068795919418}]


In [15]:
# Prueba con otro ejemplo nuevo:
print('Finetuned', finetuned_sentiment_classifier("Why is my payment still pending?")[0])
print('Original', sentiment_classifier("Why is my payment still pending?")[0])

Finetuned [{'label': 'NEGATIVE', 'score': 0.9853620529174805}, {'label': 'POSITIVE', 'score': 0.014637909829616547}]
Original [{'label': 'NEGATIVE', 'score': 0.9983781576156616}, {'label': 'POSITIVE', 'score': 0.0016218503005802631}]


In [16]:
# Prueba con el conjunto de datos de evaluación:
for reg in eval_dataset:
    result = finetuned_sentiment_classifier(reg["text"])
    print(reg["text"], result[0])

Reasons for top up cancellation. [{'label': 'NEGATIVE', 'score': 0.9972249269485474}, {'label': 'POSITIVE', 'score': 0.0027751270681619644}]
You can use it anywhere that accepts Mastercard. [{'label': 'NEGATIVE', 'score': 0.0024649619590491056}, {'label': 'POSITIVE', 'score': 0.997534990310669}]
the transfer to an account wasn't allowed is there any other way for me to do it [{'label': 'NEGATIVE', 'score': 0.6479203104972839}, {'label': 'POSITIVE', 'score': 0.35207968950271606}]
Are extra charges added for sending out additional cards? [{'label': 'NEGATIVE', 'score': 0.9633269906044006}, {'label': 'POSITIVE', 'score': 0.03667300194501877}]
How can I change my address? [{'label': 'NEGATIVE', 'score': 0.9985414743423462}, {'label': 'POSITIVE', 'score': 0.0014585881726816297}]
Can I top up my card with other cards? [{'label': 'NEGATIVE', 'score': 0.564570426940918}, {'label': 'POSITIVE', 'score': 0.43542954325675964}]
I've been checking my statement to see if I've received a refund I requ

## 🔬 Fine-tuning con el conjunto de datos de entrenamiento ampliado

Añadiendo los nuevos ejemplos al conjunto de datos de entrenamiento y creando una nueva versión del clasificador de sentimientos.

In [17]:
# Tokenización del conjunto de datos original y concatenación con el conjunto de datos tokenizado:
tokenized_train_ds_3 = train_ds.map(tokenize_function, batched=True)
train_dataset_3 = concatenate_datasets([train_dataset, tokenized_train_ds_3])

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [18]:
# Mezcla del conjunto de datos ampliado:
train_ds_3 = train_dataset_3.shuffle(seed=42)

# Configuración del entrenador con el conjunto de datos ampliado:
trainer_3 = Trainer(
    args=training_args,
    model=model,
    train_dataset=train_dataset_3,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

# Entrenamiento del modelo con el conjunto de datos ampliado:
trainer_3.train()
model.save_pretrained("distilbert-base-uncased-sentiment-banking")

/Users/edprata/anaconda3/envs/Python-3-13-2/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.000000,0.000000
2,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000


/Users/edprata/anaconda3/envs/Python-3-13-2/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/edprata/anaconda3/envs/Python-3-13-2/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/edprata/anaconda3/envs/Python-3-13-2/lib/python3.13/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


## 🎯 Conclusión

El trabajo fue muy enriquecedor, ya que ofreció la oportunidad de personalizar un modelo y añadir datos al conjunto de datos. Sin embargo, resultó bastante desafiante, ya que el tutorial proporcionado correspondía a una herramienta que ya no existe (Rubrix), que había sido reemplazada (Argilla), y también porque la documentación de ambas herramientas está bastante desactualizada. Fueron necesarias muchas personalizaciones para lograr el objetivo, pero se completó con éxito.

Otro desafío fue la capacidad de procesamiento necesaria. Inicialmente, se utilizó Google Colabority, pero su rendimiento fue insuficiente. Esto provocó retrasos en el desarrollo y las pruebas de la solución. Posteriormente, se utilizó una máquina local con mejores recursos. Sin embargo, el entrenamiento requiere una capacidad de procesamiento considerable, y la máquina se volvió extremadamente lenta y se bloqueó durante todo el proceso. Para garantizar una finalización oportuna, solo se utilizó una parte de la muestra (limitada a 100 elementos del conjunto de datos). El entrenamiento completo tomaría horas, lo cual era poco práctico durante el desarrollo. Para una aplicación real, sería necesario buscar mejores recursos para realizar el entrenamiento completo.